<a href="https://colab.research.google.com/github/subhramit/csa2022/blob/main/Res34_cifar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
transform_train= transforms.Compose([
    transforms.RandomCrop(32,padding=2),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),])
transform_test= transforms.Compose([  
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))])


train_set = torchvision.datasets.CIFAR10('./datasets', train=True,download=True, transform=transform_train)
test_set = torchvision.datasets.CIFAR10('./datasets', train=False,download=True, transform=transform_test)


train_loader = torch.utils.data.DataLoader(train_set, batch_size=128,shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=128,shuffle=False, num_workers=4)


Extracting ./datasets/cifar-10-python.tar.gz to ./datasets
Files already downloaded and verified


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(out_channels)
        )
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != self.expansion * out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, self.expansion * out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * out_channels),
            )
    def forward(self, x):
        out = self.features(x)
        out += self.shortcut(x)
        out = torch.relu(out)
        return out

In [ ]:
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True)
        )
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.avg_pool = nn.AvgPool2d(kernel_size=4)
        self.classifer = nn.Linear(512 * block.expansion, num_classes)
        
    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels * block.expansion
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.features(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.classifer(out)
        return out

this is block repetition for res34


In [ ]:
resnet=ResNet(BasicBlock, [3,4,6,3]).to(device)
resnet

In [ ]:
lr = 1e-1
loss_func= nn.CrossEntropyLoss()
optimiser= torch.optim.Adam(resnet.parameters(),lr=lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimiser, threshold=0.1)

In [ ]:

def train(epoch,):
    print('\nEpoch: %d' % (epoch))
    resnet.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimiser.zero_grad()
        outputs = resnet(inputs)
        loss = loss_func(outputs, targets)
        loss.backward()
        optimiser.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        if batch_idx % 100 == 99:    
            print('[%d, %5d] loss: %.6f |  Acc: %.3f%% (%d/%d)' %
                  (epoch + 1, batch_idx + 1, train_loss, 100.*correct/total, correct, total))
    return train_loss

In [ ]:
for epoch in range(0,50):
    loss = train(epoch)
    print('Total loss: %.6f' % loss)
    scheduler.step(loss, epoch=epoch)


Epoch: 0


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


[1,   100] loss: 185.626995 |  Acc: 31.352% (4013/12800)
[1,   200] loss: 352.982768 |  Acc: 34.527% (8839/25600)
[1,   300] loss: 508.687939 |  Acc: 37.195% (14283/38400)
Total loss: 641.935596

Epoch: 1


/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:628: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


[2,   100] loss: 139.121776 |  Acc: 49.000% (6272/12800)
[2,   200] loss: 271.006103 |  Acc: 50.195% (12850/25600)
[2,   300] loss: 396.693378 |  Acc: 51.557% (19798/38400)
Total loss: 507.040579

Epoch: 2
[3,   100] loss: 112.775806 |  Acc: 59.336% (7595/12800)
[3,   200] loss: 219.744841 |  Acc: 60.547% (15500/25600)
[3,   300] loss: 324.024507 |  Acc: 61.156% (23484/38400)
Total loss: 414.204898

Epoch: 3
[4,   100] loss: 93.073471 |  Acc: 66.734% (8542/12800)
[4,   200] loss: 183.828107 |  Acc: 67.125% (17184/25600)
[4,   300] loss: 274.379137 |  Acc: 67.612% (25963/38400)
Total loss: 351.963298

Epoch: 4
[5,   100] loss: 81.128208 |  Acc: 71.688% (9176/12800)
[5,   200] loss: 159.045742 |  Acc: 72.066% (18449/25600)
[5,   300] loss: 236.024195 |  Acc: 72.315% (27769/38400)
Total loss: 303.674363

Epoch: 5
[6,   100] loss: 68.994550 |  Acc: 75.898% (9715/12800)
[6,   200] loss: 136.533165 |  Acc: 76.180% (19502/25600)
[6,   300] loss: 203.389879 |  Acc: 76.430% (29349/38400)
Total 

In [ ]:
dataiter = iter(test_loader)
images, labels = dataiter.next()
outputs = resnet(images.to(device))
_, predicted = torch.max(outputs.cpu(), 1)
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = resnet(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Accuracy of the network on the 10000 test images: 86 %


In [ ]:
from torchsummary import summary
summary(resnet,(3,32,32))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
       BatchNorm2d-2           [-1, 64, 32, 32]             128
              ReLU-3           [-1, 64, 32, 32]               0
            Conv2d-4           [-1, 64, 32, 32]          36,864
       BatchNorm2d-5           [-1, 64, 32, 32]             128
              ReLU-6           [-1, 64, 32, 32]               0
            Conv2d-7           [-1, 64, 32, 32]          36,864
       BatchNorm2d-8           [-1, 64, 32, 32]             128
        BasicBlock-9           [-1, 64, 32, 32]               0
           Conv2d-10           [-1, 64, 32, 32]          36,864
      BatchNorm2d-11           [-1, 64, 32, 32]             128
             ReLU-12           [-1, 64, 32, 32]               0
           Conv2d-13           [-1, 64, 32, 32]          36,864
      BatchNorm2d-14           [-1, 64,